In [5]:
# 1. Import library
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Input, BatchNormalization, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab.patches import cv2_imshow 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import cv2 as cv
import random
import time
import glob

# 2. Define model and add noise function
def AEfunction(input_shape=(128,128,3)):
  
  # ENCODER
  X_input = Input(input_shape)
  
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(X_input)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2), padding='same')(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  encoded = MaxPooling2D((2, 2), padding='same',name='encoder')(x)

  # DECODER
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
  x = BatchNormalization()(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = UpSampling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = UpSampling2D((2, 2))(x)
  decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
  
  model = Model(inputs = X_input, outputs = decoded, name='Autoencoder')
  
  return model

def noisy(x):
  noise = np.random.normal(loc=0.0, scale=0.1, size=x.shape)
  x = x + noise
  x = x.clip(0.,1.)
  
  return x

# 3. Load data and add noise
BATCH_SIZE = 128
datagen = ImageDataGenerator(rescale=1./255,
                             preprocessing_function=noisy)
X_train_noisy = datagen.flow_from_directory("/content/drive/MyDrive/Colab/ReID/Dataset/Train",
                                      target_size=(128, 128),
                                      batch_size=BATCH_SIZE,
                                      class_mode='input')
X_test_noisy = datagen.flow_from_directory("/content/drive/MyDrive/Colab/ReID/Dataset/Test",
                                           target_size=(128, 128),
                                           batch_size=BATCH_SIZE,
                                           class_mode='input')

Found 16522 images belonging to 1 classes.
Found 17661 images belonging to 1 classes.


In [6]:
# 4. Define model
autoencoder = AEfunction(input_shape = (128, 128, 3))

# 5. Compile model
autoencoder.compile(loss='mse',
              optimizer='adam')

# 6. Train with data

step_size_train = X_train_noisy.samples//BATCH_SIZE
step_size_valid = X_test_noisy.samples//BATCH_SIZE

H = autoencoder.fit(X_train_noisy,
                    steps_per_epoch = step_size_train,
                    validation_data= X_test_noisy,
                    validation_steps = step_size_valid,
                    epochs = 3,
                    verbose= 1)

Epoch 1/3
129/129 [==============================] - 3597s 28s/step - loss: 0.0582 - val_loss: 0.0012
Epoch 2/3
129/129 [==============================] - 155s 1s/step - loss: 0.0100 - val_loss: 1.3036e-04
Epoch 3/3
129/129 [==============================] - 153s 1s/step - loss: 0.0067 - val_loss: 0.0015


In [10]:
autoencoder.save("/content/drive/MyDrive/Colab/ReID/autoencoder.h5")
print("Saved")

Saved
